In [6]:
import numpy as np 
import pandas as pd 

In [124]:
deliveries=pd.read_csv("deliveries.csv")
player=pd.read_csv("Player.csv")
player_match=pd.read_csv("Player_Match.csv")
runs=deliveries.groupby(["match_id","batter"])["batsman_runs"].sum().reset_index()
balls=deliveries.groupby(["match_id","batter"])["batsman_runs"].count().reset_index()
fours=deliveries[deliveries["batsman_runs"]==4].groupby(["match_id","batter"])["batsman_runs"].count().reset_index()
six=deliveries[deliveries["batsman_runs"]==6].groupby(["match_id","batter"])["batsman_runs"].count().reset_index()
df=runs.merge(balls,on=["match_id","batter"],suffixes=["_runs","_balls"]).merge(fours,on=["match_id","batter"],how="left").merge(six,on=["match_id","batter"],how="left")
df["strike_rate"]=(df["batsman_runs_runs"]/df["batsman_runs_balls"])*100
df.rename(columns={"batsman_runs_runs":"runs","batsman_runs_balls":"balls","batsman_runs_x":"fours","batsman_runs_y":"six"},inplace=True)
player=player.merge(player_match,on=["Player_Id"])[["Player_Id","Player_Name","Is_Captain"]]
df=df.merge(player,left_on="batter",right_on="Player_Name").fillna(0).drop_duplicates(subset=["match_id","batter"],keep="first").drop(columns=["Player_Id","Player_Name"])
def dream11(row):
    points=0
    points=points+row["runs"]+4*row["fours"]+6*row["six"]
    if row["runs"]>=25 and row["runs"]<50:
        points=points+4
    elif row["runs"]>=50 and row["runs"]<75:
        points=points+8
    elif row["runs"]>=75 and row["runs"]<100:
        points=points+12
    elif row["runs"]>=100:
        points=points+16
    elif row["runs"]==0:
        points=points-2
    if row["balls"]>=10:
        if row["strike_rate"]>=170:
            points=points+6
        elif row["strike_rate"]>=150.01 and row["strike_rate"]<170:
            points=points+4
        elif row["strike_rate"]>=130 and row["strike_rate"]<150:
            points=points+2
        elif row["strike_rate"]>=60 and row["strike_rate"]<70:
            points=points-2
        elif row["strike_rate"]>=50 and row["strike_rate"]<59.99:
            points=points-4
        else:
            points=points-6
    if row["Is_Captain"]==1:
        points=points*2
    return points
df["dream11_points"]=df.apply(dream11,axis=1)
df=df[["match_id","batter","dream11_points"]]